In [1]:
from convert_sd_to_diffusers import *

/storage/home/lanzhenzhongLab/public/miniconda3/envs/ll_sdtrain/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
original_config_file = "/storage/home/lanzhenzhongLab/liling/printidea/stable-diffusion-finetune/configs/based/laionart-8gpu.yaml"
original_config = OmegaConf.load(original_config_file)

In [3]:
checkpoint_path="/storage/home/lanzhenzhongLab/liling/printidea/stable-diffusion-webui/models/Stable-diffusion/model_20221213022628.ckpt"
checkpoint = torch.load(checkpoint_path, map_location="cpu")["state_dict"]

In [5]:
# checkpoint_ema = copy_ema_weights(checkpoint, original_config)

In [6]:
unet_config = create_unet_diffusers_config(original_config)
converted_unet_checkpoint = convert_ldm_unet_checkpoint(checkpoint, unet_config)

In [7]:
unet = UNet2DConditionModel(**unet_config)
unet.load_state_dict(converted_unet_checkpoint)

<All keys matched successfully>

In [ ]:
# Convert the VAE model.
vae_config = create_vae_diffusers_config(original_config)
converted_vae_checkpoint = convert_ldm_vae_checkpoint(checkpoint, vae_config)

In [13]:
vae_path = "/storage/home/lanzhenzhongLab/liling/printidea/image_gen/stable-diffusion-animate/models/Stable-diffusion/novelailatest-pruned.vae.pt"
checkpoint_vae = torch.load(vae_path, map_location="cpu")["state_dict"]
checkpoint_vae = copy_ema_weights(checkpoint_vae, original_config)

LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
Keeping EMAs of 688.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at /storage/home/lanzhenzhongLab/liling/models/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.23.mlp.fc1.weight', 'vision_model.encoder.layers.23.self_attn.q_proj.weight', 'vision_model.encoder.layers.6.mlp.fc2.bias', 'vision_model.encoder.layers.19.self_attn.out_proj.weight', 'vision_model.encoder.layers.19.self_attn.q_proj.bias', 'vision_model.encoder.layers.21.self_attn.v_proj.weight', 'vision_model.encoder.layers.9.self_attn.out_proj.weight', 'vision_model.encoder.layers.14.self_attn.k_proj.weight', 'vision_model.encoder.layers.12.layer_norm1.bias', 'vision_model.encoder.layers.16.layer_norm2.bias', 'vision_model.encoder.layers.14.layer_norm1.weight', 'vision_model.encoder.layers.7.mlp.fc1.weight', 'vision_model.encoder.layers.0.self_attn.out_proj.bias', 'vision_model.encoder.layers.20.layer_norm2.bias', 'vision_model.encoder.layers.1.self_attn.v_proj.bias', 'vision_model.encoder.layers.8.self

In [15]:
keys = list(checkpoint_vae.keys())
print(keys)

['encoder.conv_in.weight', 'encoder.conv_in.bias', 'encoder.down.0.block.0.norm1.weight', 'encoder.down.0.block.0.norm1.bias', 'encoder.down.0.block.0.conv1.weight', 'encoder.down.0.block.0.conv1.bias', 'encoder.down.0.block.0.norm2.weight', 'encoder.down.0.block.0.norm2.bias', 'encoder.down.0.block.0.conv2.weight', 'encoder.down.0.block.0.conv2.bias', 'encoder.down.0.block.1.norm1.weight', 'encoder.down.0.block.1.norm1.bias', 'encoder.down.0.block.1.conv1.weight', 'encoder.down.0.block.1.conv1.bias', 'encoder.down.0.block.1.norm2.weight', 'encoder.down.0.block.1.norm2.bias', 'encoder.down.0.block.1.conv2.weight', 'encoder.down.0.block.1.conv2.bias', 'encoder.down.0.downsample.conv.weight', 'encoder.down.0.downsample.conv.bias', 'encoder.down.1.block.0.norm1.weight', 'encoder.down.1.block.0.norm1.bias', 'encoder.down.1.block.0.conv1.weight', 'encoder.down.1.block.0.conv1.bias', 'encoder.down.1.block.0.norm2.weight', 'encoder.down.1.block.0.norm2.bias', 'encoder.down.1.block.0.conv2.wei

In [17]:
from convert_sd_to_diffusers import *
# Convert the VAE model.
vae_config = create_vae_diffusers_config(original_config)
converted_vae_checkpoint = convert_ldm_vae_checkpoint_v2(checkpoint_vae, vae_config)

NameError: name 'convert_ldm_vae_checkpoint_v2' is not defined

In [10]:
vae = AutoencoderKL(**vae_config)
vae.load_state_dict(converted_vae_checkpoint)

<All keys matched successfully>

In [11]:
# Convert the text model.
text_model_type = original_config.model.params.cond_stage_config.target.split(".")[-1]
if text_model_type == "FrozenCLIPEmbedder":
    text_model = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
else:
    # TODO: update the convert function to use the state_dict without the model instance.
    text_config = create_ldm_bert_config(original_config)
    text_model = convert_ldm_bert_checkpoint(checkpoint, text_config)
    tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.23.mlp.fc1.weight', 'vision_model.encoder.layers.23.self_attn.q_proj.weight', 'vision_model.encoder.layers.6.mlp.fc2.bias', 'vision_model.encoder.layers.19.self_attn.out_proj.weight', 'vision_model.encoder.layers.19.self_attn.q_proj.bias', 'vision_model.encoder.layers.21.self_attn.v_proj.weight', 'vision_model.encoder.layers.9.self_attn.out_proj.weight', 'vision_model.encoder.layers.14.self_attn.k_proj.weight', 'vision_model.encoder.layers.12.layer_norm1.bias', 'vision_model.encoder.layers.16.layer_norm2.bias', 'vision_model.encoder.layers.14.layer_norm1.weight', 'vision_model.encoder.layers.7.mlp.fc1.weight', 'vision_model.encoder.layers.0.self_attn.out_proj.bias', 'vision_model.encoder.layers.20.layer_norm2.bias', 'vision_model.encoder.layers.1.self_attn.v_proj.bias', 'vision_model.encoder.layers.8.self_attn.v_proj.bias', 'vision_model.enc

NameError: name 'args' is not defined

In [12]:
dump_path = "/storage/home/lanzhenzhongLab/liling/printidea/image_gen/modelweights/novelai"
safety_checker = StableDiffusionSafetyChecker.from_pretrained("CompVis/stable-diffusion-safety-checker")
feature_extractor = CLIPFeatureExtractor.from_pretrained("openai/clip-vit-large-patch14")
scheduler = create_diffusers_schedular(original_config)
pipe = StableDiffusionPipeline(
    vae=vae,
    text_encoder=text_model,
    tokenizer=tokenizer,
    unet=unet,
    scheduler=scheduler,
    safety_checker=safety_checker,
    feature_extractor=feature_extractor,
)
pipe.save_pretrained(dump_path)